### Laboratory made by:

- Ignacio Cano Navarro
- Angel Langdon Villamayor

In [5]:
!pip install transformers numpy torch sklearn emoji


     |████████████████████████████████| 2.1MB 16.7MB/s 
     |████████████████████████████████| 133kB 42.8MB/s 
     |████████████████████████████████| 3.3MB 50.5MB/s 
     |████████████████████████████████| 901kB 53.3MB/s 


In [ ]:
#!rm -rf results

# Toxicity

In this notebook, we're going to test how well BERT is able to predict toxicity and toxicity_levels variables. Lets start with toxicity 

In [6]:
import random


from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
import pandas as pd
import tensorflow as tf
import torch
from sklearn.model_selection import train_test_split
from transformers import (AutoModel, AutoModelForSequenceClassification,
                          AutoTokenizer, BertForSequenceClassification,
                          BertTokenizerFast, BertTokenizer,Trainer,
                          TrainingArguments)
from scipy.special import softmax
from transformers.file_utils import (is_tf_available, is_torch_available,
                                     is_torch_tpu_available)

Let's make a function to set a seed so we'll have same results in different runs:



In [8]:
def set_seed(seed: int):
    """
    Helper function for reproducible behavior to set the seed in ``random``, ``numpy``, ``torch`` and/or ``tf`` (if
    installed).

    Args:
        seed (:obj:`int`): The seed to set.
    """
    random.seed(seed)
    np.random.seed(seed)
    if is_torch_available():
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        # ^^ safe to call this function even if cuda is not available
    if is_tf_available():
        import tensorflow as tf

        tf.random.set_seed(seed)

set_seed(1)


we'll be using bert-base-spanish-cased because we found that the cased version had a bit more f1-score than the uncased version. Also we found that the spanish bert base pretrained model worked better than the bert base version (which absolutely makes sense) 

In [9]:
model_name = "dccuchile/bert-base-spanish-wwm-cased"
tokenizer = BertTokenizerFast.from_pretrained(model_name, do_lower_case=False)


  


Data loading and split in train/test

In [10]:
df = pd.read_csv("train.csv")
#df_test = pd.read_csv("test.csv")
train_text = df["comment"].values
train_labels = df["toxicity"].values
train_texts, valid_texts, train_labels, valid_labels = train_test_split(list(train_text),
                                                                     list(train_labels),
                                                                     test_size = 0.1)
texts = list(df["comment"])

In [11]:
# max sequence length will be the average length of texts

leng = [len(txt) for txt in texts]
max_length = sum(leng)//len(leng)
max_length


206

Now, lets convert our text to sequences of tokens

In [12]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=max_length)
valid_encodings = tokenizer(valid_texts, truncation=True, padding=True, max_length=max_length)


The below code wraps our tokenized text data into a torch Dataset



In [13]:
class DetoxisDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)

In [14]:

# convert our tokenized data into a torch Dataset
train_dataset = DetoxisDataset(train_encodings, train_labels)
valid_dataset = DetoxisDataset(valid_encodings, valid_labels)


Now that we have our data prepared, let's download and load our BERT model and its pre-trained weights. Right now we're just trying to predict whether a comment is toxic or not so the number of labels is just 2.

In [15]:
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2).to("cuda")


Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchi

In [16]:
from sklearn.metrics import accuracy_score

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  # calculate accuracy using sklearn's function
  acc = accuracy_score(labels, preds)
  return {'accuracy': acc,}


After loading the model, lets choose the training parameters for our model.

In [17]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=4,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=20,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
    # but you can specify `metric_for_best_model` argument to change to accuracy or other metric
    logging_steps=200,               # log & save weights each logging_steps
    evaluation_strategy="steps",     # evaluate each `logging_steps`
    learning_rate = 0.00001
)


In [18]:
trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=valid_dataset,          # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
)


In [19]:
# train the model
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,Runtime,Samples Per Second
200,0.655600,0.633793,0.634006,4.735400,73.278000
400,0.516700,0.567930,0.726225,4.831500,71.820000
600,0.412500,0.577231,0.726225,4.892500,70.925000


TrainOutput(global_step=780, training_loss=0.4670023453541291, metrics={'train_runtime': 572.3492, 'train_samples_per_second': 1.363, 'total_flos': 1692331864908672.0, 'epoch': 4.0, 'init_mem_cpu_alloc_delta': 0, 'init_mem_gpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 7540736, 'train_mem_gpu_alloc_delta': 1777395200, 'train_mem_cpu_peaked_delta': 8454144, 'train_mem_gpu_peaked_delta': 2955486208})

In [20]:
trainer.evaluate()


{'epoch': 4.0,
 'eval_accuracy': 0.7262247838616714,
 'eval_loss': 0.567929744720459,
 'eval_mem_cpu_alloc_delta': 0,
 'eval_mem_cpu_peaked_delta': 0,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_gpu_peaked_delta': 170200064,
 'eval_runtime': 4.7068,
 'eval_samples_per_second': 73.724}

After training our model, we're going to create a function that will receive a text as input and will return the model's prediction (toxic or not) as output.

In [21]:
def get_prediction(text,max_length):
    # prepare our text into tokenized sequence
    inputs = tokenizer(text, padding=True, truncation=True, max_length=max_length, return_tensors="pt").to("cuda")
    # perform inference to our model
    outputs = model(**inputs)
    # get output probabilities by doing softmax
    probs = outputs[0].softmax(1)
    # executing argmax function to get the candidate label
    return probs.argmax()


In [22]:
from sklearn.metrics import f1_score
pred = [int(get_prediction(text, max_length)) for text in valid_texts]
score = f1_score(valid_labels, pred, average='macro')
print(score)

0.6872075182201764


We can see that the F1 is around 0.7 which we don't think is that much improvement, taking into account that a much simpler model such as SVM was able to have a F1 score of ~0.68 and that BERT uses lots of resources and its a lot more time consuming. Also, this is the best pretrained version of BERT that we could find, other versions such as Distile-BERT or BERT-BASE, GPT-2, BETO-Cased , twitter-roberta-base-offensive were even worse than a SVM with TFIDF matrix as input.

## Toxicity_levels

Now we're going to test how well BERT predicts the variable toxicity_levels (we expect it to be much worse than the variable toxicity)

In [23]:
df = pd.read_csv("train.csv")
#df_test = pd.read_csv("test.csv")
train_text = df["comment"].values
train_labels = df["toxicity_level"].values
train_texts, valid_texts, train_labels, valid_labels = train_test_split(list(train_text),
                                                                     list(train_labels),
                                                                     test_size = 0.1)
texts = list(df["comment"])

In [24]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=max_length)
valid_encodings = tokenizer(valid_texts, truncation=True, padding=True, max_length=max_length)


train_dataset = DetoxisDataset(train_encodings, train_labels)
valid_dataset = DetoxisDataset(valid_encodings, valid_labels)


In [25]:
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=4).to("cuda")


Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchi

In [26]:
trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=valid_dataset,          # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
)

# train the model
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,Runtime,Samples Per Second
200,1.001900,0.902090,0.648415,4.817100,72.035000
400,0.781200,0.784896,0.665706,4.886200,71.016000
600,0.661800,0.734273,0.688761,4.851200,71.528000


TrainOutput(global_step=780, training_loss=0.741595224233774, metrics={'train_runtime': 577.9182, 'train_samples_per_second': 1.35, 'total_flos': 1692355558573824.0, 'epoch': 4.0, 'init_mem_cpu_alloc_delta': 0, 'init_mem_gpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': -42553344, 'train_mem_gpu_alloc_delta': 1777237504, 'train_mem_cpu_peaked_delta': 42561536, 'train_mem_gpu_peaked_delta': 2955480064})

In [30]:
from sklearn.metrics import f1_score
pred = [int(get_prediction(text, max_length)) for text in valid_texts]
score = f1_score(valid_labels, pred, average='macro')
print(score)

0.3575733859164092


## Conclusion

As we expected the f1-score when predicting toxicity_level (0.44) is much worse than toxicity(0.7). We also think that it may not be worth it in this case to use BERT over much simpler methods due to the small improvement compared to the cost of using it. In production it would be much easier and simpler to just use a regression technique or SVM. We think that this is due to the lack of training corpues ~3k tweets is quite a small dataset and our models can't generalize and thus the F1-score (and also the accuracy) is not as good as we would like it to be.